강의에선 안 다루지만 주피터 노트북이 강의별로 분리되어 있기에 이 파일이 필요하다고 판단하여 사용합니다.

In [17]:
import sys
from pathlib import Path

ROOT = Path(".").resolve()   # notebooks/ 동일 폴더
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from kis_nine import *

## Step 10-0. ETF 포트폴리오 데이터 저장 디렉터리

이 강의에서는 ETF 구성종목 및 섹터/시가총액 정보를
매일 1번 배치로 수집해서 파일로 저장합니다.

저장 구조는 다음과 같이 나눕니다.

- `data/etf_components/`:
  - ETF별, 기준일별 **구성 종목 상세 스냅샷**
  - 예: `data/etf_components/069500_2024-01-15_components.parquet`

- `data/etf_portfolio_summary/`:
  - ETF별, 기준일별 **요약 지표**
  - 예: `data/etf_portfolio_summary/069500_2024-01-15_summary.parquet`

회전율(지표 14)은
- `etf_components`에 저장된 "어제 vs 오늘 비중"을 비교해서 계산할 것이므로,
- 여기서는 "스냅샷을 잘 저장하는 것"까지 구현합니다.

In [18]:
from datetime import date
import time

ETF_COMPONENT_DIR = DATA_DIR / "etf_components"
ETF_SUMMARY_DIR   = DATA_DIR / "etf_portfolio_summary"

ETF_COMPONENT_DIR.mkdir(parents=True, exist_ok=True)
ETF_SUMMARY_DIR.mkdir(parents=True, exist_ok=True)

print("ETF_COMPONENT_DIR:", ETF_COMPONENT_DIR)
print("ETF_SUMMARY_DIR  :", ETF_SUMMARY_DIR)

ETF_COMPONENT_DIR: /Users/ulift/workspace/py-etf-mango/script2/data/etf_components
ETF_SUMMARY_DIR  : /Users/ulift/workspace/py-etf-mango/script2/data/etf_portfolio_summary


## Step 10-1. 공통 KIS GET 호출 유틸

앞으로 KIS REST API를 호출할 때는
- `call_kis_get(path, tr_id, params)` 한 함수만 사용합니다.

이 함수는:
1. `ensure_kis_token()`으로 토큰이 있는지 확인하고
2. 실제 GET 요청을 보낸 뒤
3. 응답의 `rt_cd`를 확인해서
   - 정상(`"0"`)이면 JSON을 그대로 반환
   - 만료 코드(`EGW00123`, `EGW00121`)면 `refresh_access_token()` 후 한 번 더 재시도
   - 그 외에는 `RuntimeError`를 발생시켜 로그 상에서 바로 문제를 찾을 수 있게 합니다.

이 패턴을 만들어 두면:
- ETF 구성종목 API
- 주식기본조회 API
- 그 외 추가되는 모든 API
가 동일한 방식으로 관리됩니다.

In [19]:
from typing import Any, Dict
import requests

def call_kis_get(
    path: str,
    tr_id: str,
    params: Dict[str, Any],
    *,
    max_retry: int = 1,
) -> Dict[str, Any]:
    """
    KIS GET 공통 헬퍼.

    - path: 'uapi/...' 형태의 엔드포인트 경로
    - tr_id: KIS 문서 상의 TR ID (예: FHKST121600C0)
    - params: 쿼리 파라미터 dict
    - max_retry: 토큰 만료 시 재시도 횟수

    반환:
    - KIS 응답 JSON(dict). rt_cd != "0"인 경우 RuntimeError 발생.
    """
    if not KIS_URL_BASE:
        raise RuntimeError("KIS_URL_BASE가 설정되어 있지 않습니다. .env를 확인하세요.")

    # 0) 토큰이 하나는 존재하도록 보장
    ensure_kis_token()

    url = f"{KIS_URL_BASE.rstrip('/')}/{path.lstrip('/')}"

    for attempt in range(max_retry + 1):
        headers = get_api_headers(tr_id)

        try:
            res = requests.get(url, headers=headers, params=params, timeout=5)
            res.raise_for_status()
            data = res.json()
        except Exception as e:
            raise RuntimeError(f"KIS GET 실패 ({path}, 시도={attempt+1}): {e}") from e

        rt_cd = data.get("rt_cd")
        if rt_cd == "0":
            return data

        # 토큰 만료 코드인 경우, 토큰 재발급 후 재시도
        if rt_cd in ("EGW00123", "EGW00121") and attempt < max_retry:
            print(f"[INFO] 토큰 만료(rt_cd={rt_cd}), refresh 후 재시도...")
            refresh_access_token()
            continue

        # 여기까지 왔다는 것은 재시도 후에도 실패한 경우
        msg = data.get("msg1") or data.get("msg_cd") or "알 수 없는 오류"
        raise RuntimeError(f"KIS API 오류 (path={path}, rt_cd={rt_cd}, msg={msg})")

    # 논리상 도달하지 않지만, 안전망
    raise RuntimeError(f"KIS GET 실패: 최대 재시도({max_retry})를 초과했습니다. path={path}")

## Step 10-2. ETF 구성종목 시세 조회 (`FHKST121600C0`)

- 사용 API: "ETF 구성종목시세"
  - PATH: `/uapi/etfetn/v1/quotations/inquire-component-stock-price`
  - TR_ID: `FHKST121600C0`
- 주요 필드 (output2 기준):
  - `stck_shrn_iscd`: 구성종목 코드 (6자리)
  - `hts_kor_isnm`: 한글 종목명
  - `stck_prpr`: 현재가
  - `etf_vltn_amt`: 해당 종목의 ETF 내 평가금액

여기서:
- 각 종목별 비중(weight)을 계산하고
- 상위 10개 종목의 집중도(비중 합)를 함께 반환합니다.

In [20]:
import pandas as pd

def fetch_etf_components_raw(etf_code: str) -> pd.DataFrame:
    """
    KIS 'ETF 구성종목시세' API를 호출하여
    ETF 구성 종목 리스트와 주요 수치들을 DataFrame으로 반환.

    반환되는 DataFrame 주요 컬럼:
    - as_of_date: 스냅샷 기준일 (오늘 날짜, YYYY-MM-DD)
    - etf_code: ETF 6자리 코드
    - stck_shrn_iscd: 구성 종목 코드
    - hts_kor_isnm: 종목명
    - stck_prpr: 현재가 (원본 문자열)
    - etf_vltn_amt: 평가금액 (원본 문자열)
    - weight: ETF 내 비중 (0~1 실수)
    - etf_vltn_amt_float: 평가금액(숫자형)
    기타 API에서 내려주는 원본 필드들도 모두 포함됩니다.
    """
    path = "/uapi/etfetn/v1/quotations/inquire-component-stock-price"
    tr_id = "FHKST121600C0"

    params = {
        "FID_COND_MRKT_DIV_CODE": "J",      # 국내 ETF
        "FID_INPUT_ISCD": etf_code,         # ETF 종목코드(6자리)
        "FID_COND_SCR_DIV_CODE": "11216",   # 문서 예시 값
    }

    data = call_kis_get(path, tr_id, params)
    rows = data.get("output2") or []

    if not rows:
        raise RuntimeError(f"ETF 구성종목이 비어 있습니다. etf_code={etf_code}, data={data}")

    df = pd.DataFrame(rows).copy()

    # 코드/이름 정리
    df["etf_code"] = str(etf_code)
    df["stck_shrn_iscd"] = df["stck_shrn_iscd"].astype(str).str.strip().str.zfill(6)
    df["hts_kor_isnm"] = df["hts_kor_isnm"].astype(str).str.strip()

    # 숫자 필드 변환
    if "etf_vltn_amt" in df.columns:
        df["etf_vltn_amt_float"] = df["etf_vltn_amt"].apply(to_float)
    else:
        df["etf_vltn_amt_float"] = None

    if "stck_prpr" in df.columns:
        df["stck_prpr_float"] = df["stck_prpr"].apply(to_float)
    else:
        df["stck_prpr_float"] = None

    # 총 평가금액 및 비중 계산
    total_vltn = df["etf_vltn_amt_float"].fillna(0).sum()
    if total_vltn <= 0:
        print("⚠️ etf_vltn_amt 합계가 0 이하입니다. weight 계산이 불가능할 수 있습니다.")
        df["weight"] = None
    else:
        df["weight"] = df["etf_vltn_amt_float"] / total_vltn

    # 기준일(오늘 날짜) 추가
    today_str = date.today().isoformat()
    df["as_of_date"] = today_str

    # 컬럼 순서 약간 정리
    preferred_cols = [
        "as_of_date",
        "etf_code",
        "stck_shrn_iscd",
        "hts_kor_isnm",
        "stck_prpr",
        "stck_prpr_float",
        "etf_vltn_amt",
        "etf_vltn_amt_float",
        "weight",
    ]
    other_cols = [c for c in df.columns if c not in preferred_cols]
    df = df[preferred_cols + other_cols]

    return df


def calc_top10_concentration(df_components: pd.DataFrame) -> float:
    """
    구성종목 DataFrame에서 'weight' 기준 상위 10개 종목의
    비중 합계를 반환 (0~1 사이 실수).
    """
    if "weight" not in df_components.columns:
        return float("nan")

    tmp = df_components.dropna(subset=["weight"]).copy()
    if tmp.empty:
        return float("nan")

    tmp = tmp.sort_values("weight", ascending=False)
    top10 = tmp.head(10)
    top10_conc = top10["weight"].sum()
    return float(top10_conc)

## Step 10-3. ETF/ETN 현재가 + NAV/괴리율 스냅샷 (`FHPST02400000`)

- 사용 API: "ETF/ETN 현재가"
  - PATH: `/uapi/etfetn/v1/quotations/inquire-price`
  - TR_ID: `FHPST02400000`
- 주요 필드:
  - `stck_prpr`: 현재가
  - `nav`: 실시간 NAV
  - `prdy_last_nav`: 전일 최종 NAV
  - `trc_errt`: 추적오차(또는 괴리 관련 수치, 문서 정의 참조)
  - `etf_ntas_ttam`: 순자산총액(원화 기준)

여기서:
- 가격 / NAV를 숫자로 변환하고
- 단일 스냅샷 DataFrame 한 줄로 반환해서
- 포트폴리오 메타 정보(meta)에 함께 저장합니다.

In [ ]:

def fetch_etf_nav_snapshot(etf_code: str) -> pd.DataFrame:
    """
    KIS 'ETF/ETN 현재가' API를 호출하여
    NAV / 괴리율 / AUM 관련 정보를 한 줄짜리 DataFrame으로 반환.

    반환 컬럼 예시:
    - etf_code
    - stck_prpr, stck_prpr_float
    - nav, nav_float
    - prdy_last_nav, prdy_last_nav_float
    - trc_errt, trc_errt_float
    - etf_ntas_ttam, etf_ntas_ttam_float (순자산총액 근사)
    - premium_rate: (현재가 / NAV - 1.0)  → 괴리율(%는 아님, 0.01 = +1%)
    """
    path = "/uapi/etfetn/v1/quotations/inquire-price"
    tr_id = "FHPST02400000"

    etf_code = str(etf_code).strip().zfill(6)

    params = {
        "fid_input_iscd": etf_code,
        "fid_cond_mrkt_div_code": "J",  # 국내 ETF/ETN
    }

    data = call_kis_get(path, tr_id, params)
    output = data.get("output") or {}
    if not output:
        raise RuntimeError(f"ETF/ETN 현재가 output이 비어 있습니다. etf_code={etf_code}, data={data}")

    row = dict(output)
    row["etf_code"] = etf_code

    df = pd.DataFrame([row])

    # 숫자 컬럼 변환
    num_cols = [
        "stck_prpr",
        "nav",
        "prdy_last_nav",
        "trc_errt",
        "etf_ntas_ttam",
        "etf_frcr_ntas_ttam",
        "etf_crcl_ntas_ttam",
        "etf_frcr_crcl_ntas_ttam",
        "lstn_stcn",
    ]
    for col in num_cols:
        if col in df.columns:
            df[col + "_float"] = df[col].apply(to_float)
        else:
            df[col + "_float"] = None

    # 괴리율: (현재가 / NAV - 1)
    nav_val = df["nav_float"].iloc[0] if "nav_float" in df.columns else None
    price_val = df["stck_prpr_float"].iloc[0] if "stck_prpr_float" in df.columns else None

    premium = None
    if isinstance(nav_val, (int, float)) and nav_val and isinstance(price_val, (int, float)):
        try:
            premium = float(price_val / nav_val - 1.0)
        except ZeroDivisionError:
            premium = None

    df["premium_rate"] = premium  # 0.01 = +1% 괴리

    return df

## Step 10-4. 주식기본조회로 섹터/상장주수 조회 (`CTPF1002R`)

- 사용 API: "주식기본조회"
  - PATH: `/uapi/domestic-stock/v1/quotations/search-stock-info`
  - TR_ID: `CTPF1002R`
- 주요 필드:
  - `std_idst_clsf_cd_name`: 표준산업분류명 (섹터 이름)
  - `lstg_stqt`: 상장주수 (시가총액 계산용)

여기서는 여러 종목코드를 한 번에 받아서
- 각 코드에 대해 API를 개별 호출하고
- 결과를 DataFrame으로 합치는 배치 헬퍼를 만듭니다.

In [21]:
def fetch_stock_basic_batch(codes: list[str], *, sleep_sec: float = 0.05) -> pd.DataFrame:
    """
    여러 종목코드에 대해 '주식기본조회' API를 반복 호출하여
    섹터/상장주수 등 기본 정보를 모아 DataFrame으로 반환.

    인자:
    - codes: 종목코드(6자리) 리스트
    - sleep_sec: 호출 간 간격 (rate limit 방지용)

    반환 컬럼 예시:
    - stck_shrn_iscd: 종목 코드 (입력 그대로)
    - prdt_name: 종목명
    - std_idst_clsf_cd_name: 표준산업분류명 (섹터)
    - lstg_stqt: 상장주수 (원본 문자열)
    - lstg_stqt_float: 상장주수 (숫자형)
    그 외 KIS에서 내려주는 필드들도 그대로 포함됩니다.
    """
    path = "/uapi/domestic-stock/v1/quotations/search-stock-info"
    tr_id = "CTPF1002R"

    records: list[dict[str, Any]] = []

    for code in codes:
        code = str(code).strip().zfill(6)
        params = {
            "PRDT_TYPE_CD": "300",  # 주식/ETF/ETN/ELW 등
            "PDNO": code,
        }

        try:
            data = call_kis_get(path, tr_id, params)
        except Exception as e:
            print(f"⚠️ 주식기본조회 실패: code={code}, error={e}")
            continue

        output = data.get("output") or {}
        if not output:
            print(f"⚠️ 주식기본조회 output 비어 있음: code={code}")
            continue

        row = dict(output)
        row["stck_shrn_iscd"] = code
        records.append(row)

        # Rate limit 보호를 위해 약간의 sleep
        time.sleep(sleep_sec)

    if not records:
        raise RuntimeError("주식기본조회 결과가 모두 비어 있습니다. codes를 확인하세요.")

    df = pd.DataFrame(records)

    # 숫자 변환: 상장주수
    if "lstg_stqt" in df.columns:
        df["lstg_stqt_float"] = df["lstg_stqt"].apply(to_float)
    else:
        df["lstg_stqt_float"] = None

    # 문자열 정리
    for col in ["prdt_name", "std_idst_clsf_cd_name"]:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()

    return df

## Step 10-5. 섹터/시가총액 비중 계산

ETF 구성종목(df_components)과 주식기본정보(df_basic)를 병합하여:

1. 각 종목별 시가총액 = 현재가 × 상장주수
2. 시가총액을 기준으로 Large/Mid/Small 버킷을 나눈 뒤
3. 비중(weight)을 이용해:
   - 섹터별 비중 (지표 10)
   - 시가총액 버킷별 비중 (지표 12)

을 계산합니다.

In [ ]:
def classify_size_bucket(market_caps: pd.Series) -> pd.Series:
    """
    시가총액 Series를 받아서 Large/Mid/Small 버킷으로 분류.
    - 상위 30% 이상: Large
    - 하위 30% 이하: Small
    - 나머지: Mid

    *주의*: 이 버킷은 "ETF 내부 상대 크기" 기준입니다.
    """
    s = market_caps.copy()
    s = s.fillna(0)

    if len(s) == 0 or s.max() <= 0:
        return pd.Series(["Unknown"] * len(s), index=s.index)

    q_low = s.quantile(0.3)
    q_high = s.quantile(0.7)

    def _bucket(x: float) -> str:
        if x <= 0:
            return "Unknown"
        if x >= q_high:
            return "Large"
        if x <= q_low:
            return "Small"
        return "Mid"

    return s.apply(_bucket)


def build_etf_portfolio_snapshot(etf_code: str) -> dict[str, pd.DataFrame]:
    """
    하나의 ETF 코드에 대해:
    1) 구성종목 시세(df_components)를 조회하고
    2) 주식기본조회(df_basic)와 병합하여
    3) 섹터/시가총액 버킷 비중 및 Top10 집중도까지 계산한 뒤
    4) ETF 현재가/NAV/AUM 스냅샷까지 포함한 메타 정보를 생성하고
    5) 여러 DataFrame을 dict 형태로 반환.

    반환:
    {
      "components": 구성종목 상세 (비중, 섹터, 시총, Size bucket 포함),
      "sector_summary": 섹터별 비중 요약,
      "size_summary": 시가총액 버킷별 비중 요약,
      "meta": ETF 단위 메타 정보
              (Top10 집중도 + NAV + 괴리율 + AUM 근사치 등)
    }
    """
    etf_code = str(etf_code).strip().zfill(6)

    # 0) ETF 현재가/NAV/AUM 스냅샷
    df_nav = fetch_etf_nav_snapshot(etf_code)
    nav_row = df_nav.iloc[0]

    # 1) 구성종목
    df_components = fetch_etf_components_raw(etf_code)

    # 2) 주식기본조회: 유니크 종목코드만
    codes = sorted(df_components["stck_shrn_iscd"].dropna().unique().tolist())
    df_basic = fetch_stock_basic_batch(codes)

    # 3) 병합 (left join: 구성종목 기준)
    merge_cols = ["stck_shrn_iscd", "prdt_name", "std_idst_clsf_cd_name", "lstg_stqt", "lstg_stqt_float"]
    df_basic_small = df_basic[merge_cols].copy()

    df = df_components.merge(
        df_basic_small,
        on="stck_shrn_iscd",
        how="left",
        suffixes=("", "_basic"),
    )

    # 4) 시가총액 계산
    df["market_cap"] = df["stck_prpr_float"] * df["lstg_stqt_float"]

    # 5) Size bucket 분류
    df["size_bucket"] = classify_size_bucket(df["market_cap"])

    # 6) 섹터별 비중 요약 (지표 10)
    if "weight" in df.columns:
        sector_group = (
            df.dropna(subset=["weight"])
              .groupby("std_idst_clsf_cd_name", dropna=False)["weight"]
              .sum()
              .reset_index()
        )
        sector_group.rename(columns={"std_idst_clsf_cd_name": "sector", "weight": "weight_sum"}, inplace=True)
    else:
        sector_group = pd.DataFrame(columns=["sector", "weight_sum"])

    # as_of_date / etf_code 붙이기
    if not df.empty:
        as_of_date = df["as_of_date"].iloc[0]
    else:
        as_of_date = date.today().isoformat()

    sector_group["as_of_date"] = as_of_date
    sector_group["etf_code"] = etf_code

    # 7) 시가총액 버킷별 비중 요약 (지표 12)
    if "weight" in df.columns:
        size_group = (
            df.dropna(subset=["weight"])
              .groupby("size_bucket", dropna=False)["weight"]
              .sum()
              .reset_index()
        )
        size_group.rename(columns={"size_bucket": "size_bucket", "weight": "weight_sum"}, inplace=True)
    else:
        size_group = pd.DataFrame(columns=["size_bucket", "weight_sum"])

    size_group["as_of_date"] = as_of_date
    size_group["etf_code"] = etf_code

    # 8) Top10 집중도 (지표 9)
    top10_conc = calc_top10_concentration(df_components)

    # 9) AUM 근사치: 구성종목 평가금액 합
    aum_components_sum = df_components["etf_vltn_amt_float"].fillna(0).sum()

    # 10) NAV / 괴리율 / API 기반 AUM 등 메타 정보 정리
    meta = pd.DataFrame(
        [
            {
                "as_of_date": as_of_date,
                "etf_code": etf_code,
                "n_components": int(len(df)),
                "top10_concentration": float(top10_conc),

                # NAV 관련 (지표 5)
                "nav": nav_row.get("nav"),
                "nav_float": nav_row.get("nav_float"),
                "prdy_last_nav": nav_row.get("prdy_last_nav"),
                "prdy_last_nav_float": nav_row.get("prdy_last_nav_float"),

                # 괴리율 / 추적오차 관련 (지표 6)
                # premium_rate: (price/NAV - 1.0)
                "premium_rate": nav_row.get("premium_rate"),
                "trc_errt": nav_row.get("trc_errt"),
                "trc_errt_float": nav_row.get("trc_errt_float"),

                # AUM (지표 8) - 두 버전 모두 저장
                # 1) API 기반 순자산 (있으면 사용)
                "aum_official_like": nav_row.get("etf_ntas_ttam_float"),
                # 2) 구성종목 평가금액 합 (근사치)
                "aum_components_sum": float(aum_components_sum),
            }
        ]
    )

    return {
        "components": df,
        "sector_summary": sector_group,
        "size_summary": size_group,
        "meta": meta,
    }


## Step 10-6. ETF 포트폴리오 스냅샷 저장 유틸

하루 1번 배치에서 사용할 엔트리 포인트:

```python
run_etf_portfolio_job("069500")  # KODEX 200 예시
```

- 구성종목 상세 + 섹터 요약 + 시가총액 버킷 요약 + 메타 정보를
  각각 parquet 파일로 저장합니다.

In [23]:
def run_etf_portfolio_job(etf_code: str) -> dict[str, pd.DataFrame]:
    """
    ETF 포트폴리오 구성 데이터를 조회하고,
    - 구성 종목 상세
    - 섹터별 비중 요약
    - 시가총액 버킷별 비중 요약
    - 메타 정보 (Top10 집중도 등)
    를 parquet 파일로 저장한 뒤, DataFrame들을 그대로 반환.

    저장 경로 예:
    - data/etf_components/069500_2024-01-15_components.parquet
    - data/etf_portfolio_summary/069500_2024-01-15_sector.parquet
    - data/etf_portfolio_summary/069500_2024-01-15_size.parquet
    - data/etf_portfolio_summary/069500_2024-01-15_meta.parquet
    """
    etf_code = str(etf_code).strip().zfill(6)

    result = build_etf_portfolio_snapshot(etf_code)

    df_components = result["components"]
    df_sector     = result["sector_summary"]
    df_size       = result["size_summary"]
    df_meta       = result["meta"]

    if df_components.empty:
        raise RuntimeError(f"구성종목 스냅샷이 비어 있습니다. etf_code={etf_code}")

    as_of_date = df_components["as_of_date"].iloc[0]

    # 파일 경로 정의
    comp_path = ETF_COMPONENT_DIR / f"{etf_code}_{as_of_date}_components.csv"
    sector_path = ETF_SUMMARY_DIR / f"{etf_code}_{as_of_date}_sector.csv"
    size_path = ETF_SUMMARY_DIR / f"{etf_code}_{as_of_date}_size.csv"
    meta_path = ETF_SUMMARY_DIR / f"{etf_code}_{as_of_date}_meta.csv"

    # 저장
    df_components.to_csv(comp_path, index=False)
    df_sector.to_csv(sector_path, index=False)
    df_size.to_csv(size_path, index=False)
    df_meta.to_csv(meta_path, index=False)

    print(f"[SAVE] components → {comp_path}")
    print(f"[SAVE] sector_summary → {sector_path}")
    print(f"[SAVE] size_summary   → {size_path}")
    print(f"[SAVE] meta           → {meta_path}")

    return result

## Step 10-7. 예시 실행 - KODEX 200 (069500)

실제로 한 번 실행해보면:
- 구성종목 상세 1개 파일
- 섹터 요약 1개 파일
- 시가총액 버킷 요약 1개 파일
- 메타 정보 1개 파일
이 생성됩니다.

이후 강의에서:
- 오늘 파일 vs 어제 파일을 비교해서 "포트폴리오 회전율(지표 14)"를 계산하고
- 섹터/시가총액 분포를 그래프/테이블로 시각화할 예정입니다.

In [24]:
# 예시 ETF: KODEX 200
example_etf = "069500"

ensure_kis_token()  # 토큰 하나는 갖고 시작
result = run_etf_portfolio_job(example_etf)

print("\n[components] preview:")
print(result["components"].head())

print("\n[sector_summary]:")
print(result["sector_summary"])

print("\n[size_summary]:")
print(result["size_summary"])

print("\n[meta]:")
print(result["meta"])

[SAVE] components → /Users/ulift/workspace/py-etf-mango/script2/data/etf_components/069500_2025-11-28_components.csv
[SAVE] sector_summary → /Users/ulift/workspace/py-etf-mango/script2/data/etf_portfolio_summary/069500_2025-11-28_sector.csv
[SAVE] size_summary   → /Users/ulift/workspace/py-etf-mango/script2/data/etf_portfolio_summary/069500_2025-11-28_size.csv
[SAVE] meta           → /Users/ulift/workspace/py-etf-mango/script2/data/etf_portfolio_summary/069500_2025-11-28_meta.csv

[components] preview:
   as_of_date etf_code stck_shrn_iscd hts_kor_isnm stck_prpr  stck_prpr_float  \
0  2025-11-28   069500         005930         삼성전자    100500         100500.0   
1  2025-11-28   069500         000660       SK하이닉스    530000         530000.0   
2  2025-11-28   069500         105560         KB금융    124800         124800.0   
3  2025-11-28   069500         005380          현대차    261500         261500.0   
4  2025-11-28   069500         034020      두산에너빌리티     76400          76400.0   

  etf